## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Akdepe,TM,42.0551,59.3788,46.94,32,30,9.42,scattered clouds
1,1,Yabrud,SY,33.9674,36.6574,42.03,45,1,1.86,clear sky
2,2,Kaberamaido,UG,1.7389,33.1594,67.69,88,12,3.69,few clouds
3,3,Yushu,CN,44.8000,126.5333,40.57,25,3,15.37,clear sky
4,4,East London,ZA,-33.0153,27.9116,69.42,84,100,6.76,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 70
What is the maximun temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Moanda,GA,-1.5575,13.2178,71.87,88,28,1.97,scattered clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,82.63,69,10,5.84,clear sky
9,9,Belmonte,BR,-15.8631,-38.8828,76.60,83,12,3.96,few clouds
12,12,Alice Springs,AU,-23.7000,133.8833,78.35,36,37,10.36,scattered clouds
14,14,Tuy Hoa,VN,13.0833,109.3000,75.45,92,100,10.92,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

320


City_ID                320
City                   320
Country                320
Lat                    320
Lng                    320
Max Temp               320
Humidity               320
Cloudiness             320
Wind Speed             320
Current Description    320
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head(10)

320


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Moanda,GA,-1.5575,13.2178,71.87,88,28,1.97,scattered clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,82.63,69,10,5.84,clear sky
9,9,Belmonte,BR,-15.8631,-38.8828,76.60,83,12,3.96,few clouds
12,12,Alice Springs,AU,-23.7000,133.8833,78.35,36,37,10.36,scattered clouds
14,14,Tuy Hoa,VN,13.0833,109.3000,75.45,92,100,10.92,overcast clouds
16,16,Saint-Philippe,RE,-21.3585,55.7679,76.59,75,88,16.06,overcast clouds
18,18,Bengkulu,ID,-3.8004,102.2655,84.38,66,44,8.41,scattered clouds
21,21,Carnarvon,AU,-23.7000,133.8833,78.35,36,37,10.36,scattered clouds
22,22,Ambikapur,IN,23.1167,83.2000,91.51,8,9,4.76,clear sky
25,25,Maceio,BR,-9.6658,-35.7353,76.44,94,75,2.30,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Moanda,GA,71.87,scattered clouds,-1.5575,13.2178,
8,Hithadhoo,MV,82.63,clear sky,-0.6000,73.0833,
9,Belmonte,BR,76.60,few clouds,-15.8631,-38.8828,
12,Alice Springs,AU,78.35,scattered clouds,-23.7000,133.8833,
14,Tuy Hoa,VN,75.45,overcast clouds,13.0833,109.3000,
...,...,...,...,...,...,...,...
687,Charters Towers,AU,78.35,scattered clouds,-23.7000,133.8833,
688,Quatre Cocos,MU,77.67,overcast clouds,-20.5167,57.5167,
691,Juba,PH,79.27,clear sky,7.9986,124.2928,
694,Yarada,IN,91.51,clear sky,23.1167,83.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
clean_hotel_df = hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna(subset = ["Hotel Name"], axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Moanda,GA,71.87,scattered clouds,-1.5575,13.2178,Heliconia Stadium Moanda
8,Hithadhoo,MV,82.63,clear sky,-0.6000,73.0833,Scoop Guest House
9,Belmonte,BR,76.60,few clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
12,Alice Springs,AU,78.35,scattered clouds,-23.7000,133.8833,Desert Palms Alice Springs
14,Tuy Hoa,VN,75.45,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
16,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
18,Bengkulu,ID,84.38,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
21,Carnarvon,AU,78.35,scattered clouds,-23.7000,133.8833,Desert Palms Alice Springs
22,Ambikapur,IN,91.51,clear sky,23.1167,83.2000,Hotel Birendra Prabha
25,Maceio,BR,76.44,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))